# *K*-Means Clustering

$K$-means clustering is a simple and elegant approach for partitioning a data set into $K$ distinct, non-overlapping clusters. To perform $K$-means clustering, we must first specify the desired number of clusters $K$; then the $K$-means algorithm will assign each observation to exactly one of the $K$ clusters.

In [ ]:
import pandas as pd
df = pd.read_stata('http://fmwww.bc.edu/ec-p/data/wooldridge/openness.dta')

##  Obs:   114

## 1. open                     imports as % GDP, '73-
## 2. inf                      avg. annual inflation, '73-
## 3. pcinc                    1980 per capita inc., U.S. 
## 4. land                     land area, square miles
## 5. oil                      =1 if major oil producer
## 6. good                     =1 if 'good' data
## 7. lpcinc                   log(pcinc)
## 8. lland                    log(land)
## 9. lopen                    log(open)
## 10. linf                    log(inf)
## 11. opendec                 open/100
## 12. linfdec                 log(inf/100)

df.head()

The $K$-means clustering procedure results from a simple and intuitive mathematical problem.We begin by defining some notation. Let $C_1,\ldots , C_K$ denote sets containing the indices of the observations in each cluster. These sets satisfy two properties:

1. $C_{1} \cup C_{2} \cup \ldots \cup C_{K}=\{1, \ldots, n\}$. In other words, each observation belongs to at least one of the $K$ clusters.

2. $C_{k} \cap C_{k^{\prime}}=\{\emptyset\}$ for all $k \neq k^{\prime}$. In other words, the clusters are non-overlapping: no observation belongs to more than one cluster.

For instance, if the $i$th observation is in the $k$ th cluster, then $i \in C_{k}$. The idea behind $K$ -means clustering is that a good clustering is one for which the within-cluster variation is as small as possible. The within-cluster variation for cluster $C_{k}$ is a measure $W\left(C_{k}\right)$ of the amount by which the observations within a cluster differ from each other. Hence we want to solve the problem

$$
\underset{C_{1}, \ldots, C_{K}}{\operatorname{minimize}}\left\{\sum_{k=1}^{K} W\left(C_{k}\right)\right\}.
$$

In words, this formula says that we want to partition the observations into $K$ clusters such that the total within-cluster variation, summed over all $K$ clusters, is as small as possible. Solving this last equation seems like a reasonable idea, but in order to make it actionable we need to define the within-cluster variation. There are many possible ways to define this concept, but by far the most common choice involves squared Euclidean distance. That is, we define

$$
W\left(C_{k}\right)=\frac{1}{\left|C_{k}\right|} \sum_{i, i^{\prime} \in C_{k}} \sum_{j=1}^{p}\left(x_{i j}-x_{i^{\prime} j}\right)^{2},
$$

where $p=\text{dim}(x_{i j})$, and $\left|C_{k}\right|$ denotes the number of observations in the $k$ th cluster. In other words, the within-cluster variation for the $k$th cluster is the sum of all of
the pairwise squared [Euclidean distances](https://en.wikipedia.org/wiki/Euclidean_distance) between the observations in the $k$th cluster, divided by the total number of observations in the $k$th cluster. Therefore

$$
\underset{C_{1}, \ldots, C_{K}}{\operatorname{minimize}}\left\{\sum_{k=1}^{K} \frac{1}{\left|C_{k}\right|} \sum_{i, i^{\prime} \in C_{k}} \sum_{j=1}^{p}\left(x_{i j}-x_{i^{\prime} j}\right)^{2}\right\}
$$

***
**Algorithm**: <ins>_K_ Means Clustering</ins>

1. Randomly assign a number, from 1 to K, to each of the observations. These serve as initial cluster assignments for the observations.

2. Iterate until the cluster assignments stop changing:
 1. For each of the $K$ clusters, compute the cluster centroid. The $k$th cluster centroid is the vector of the $p$ feature means for the observations in the $k$th cluster.
 2. Assign each observation to the cluster whose centroid is closest (where closest is defined using Euclidean distance).
***

✍🏼 This algorithm is guaranteed to **decrease** the value of the objective function at each step.  Since the resulting classification will depend on the initial (random) cluster assignment in Step 1, the algorithm is said to find a _local_ rather than a _global_ optimum. Therefore it is important to run the algorithm multiple times from different random initial configurations, e.g., between 25 to 50 times is recommended. Then one selects the _best_ solution, i.e., that for which the objective function is the smallest.

💻 Notice that the $K$-means algorithm described here *only* works for __continuously distributed__ data. It will not work for categorical features.

💻 It is recommended to _scale_ the features prior to applying the algorithm. Different scales among features will severely affect the classification algorithm.

In [ ]:
import patsy
X = patsy.dmatrix('-1 + lopen + linf', data=df, return_type='dataframe')

<ins>**Distance Measures**</ins>: Euclidean Distance

This is the default distance measure in most ML algorithms and the one used in the explanation above. If chosen, then observations with high values of the features will be clustered together, and observations with low values of the features will also be clustered together.

⚠️ Unfortunately, `sklearn` <ins>only</ins> provides this distance measure. See this [post](https://stackoverflow.com/questions/5529625/is-it-possible-to-specify-your-own-distance-function-using-scikit-learn-k-means) to learn about alternative packages with different distance measures.

In [ ]:
from sklearn.cluster import KMeans

## Perform k-means clustering
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=20, random_state=42)
pred_y = kmeans.fit_predict(X)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(X.iloc[:,0], X.iloc[:,1],c=kmeans.labels_.astype(float)) #Plotting the data colored by cluster
plt.grid(alpha=0.4,linestyle='--') #Adding a grid
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red')
plt.show()

### Choosing *K*: The Elbow Method

The idea is that we want a small within-cluster variance, but that the within-cluster variance tends to decrease toward $0$ as we increase $K$ (the within-cluster variance is exactly $0$ when $K$ is equal to the number of data points in the dataset, because then each data point is its own cluster, and there is no error between it and the center of its cluster). So our goal is to choose a small value of $K$ that still has a low within-cluster variance, and the [elbow](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) usually represents where we start to have diminishing returns by increasing $K$.

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
plt.plot(range(1, 11), wcss)
plt.grid(alpha=0.4,linestyle='--') #Adding a grid
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()